In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, time
import numpy as np
import scipy as sp
import pandas as pd
import intake
    
from pathlib import Path
from pprint import pprint as pp
p = print 

from sklearn.externals import joblib
import pdb

import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
if not sys.warnoptions:
    warnings.simplefilter('ignore')
    
# Don't generate bytecode
sys.dont_write_bytecode = True

In [ ]:
import holoviews as hv
import xarray as xr

from holoviews import opts
from holoviews.operation.datashader import datashade, shade, dynspread, rasterize
from holoviews.streams import Stream, param
from holoviews import streams
import panel as pn

hv.notebook_extension('bokeh')
hv.Dimension.type_formatters[np.datetime64] = '%Y-%m-%d'
pn.extension()

In [ ]:
import geopandas as gpd
import cartopy.crs as ccrs
import cartopy.feature as cf

import geoviews as gv
import geoviews.feature as gf
from geoviews import tile_sources as gvts

In [ ]:
# Add the utils directory to the search path
UTILS_DIR = Path('../utils').absolute()
assert UTILS_DIR.exists()
if str(UTILS_DIR) not in sys.path:
    sys.path.insert(0, str(UTILS_DIR))
    print(f"Added {str(UTILS_DIR)} to sys.path")

pp(sys.path)

In [ ]:
from utils import get_mro as mro, nprint
import utils as u

from vector import Vector as vec
from line import Line2d

In [ ]:
# list_cmaps(provider='colorcet', category='Sequential')

In [ ]:
%opts Image [colorbar=True, tools=['hover']] Curve [tools=['hover']]

In [ ]:
# Grab registered bokeh renderer
print("Currently available renderers: ", *hv.Store.renderers.keys())
renderer = hv.renderer('bokeh')

Modified: Jul 28, 2019
 
## todo:
est: ~~30mins~~ 3days?....?
- [x] check i,j -> c,r conversion
- [ ] incorporate Grid as a data storage (self.grid) of LevelSet class
- [ ] get a list of points from holoviews polydraw
- [ ] make a list of line segments from the point list
- [ ] for each line segment, make Line2d object and get polygon for the band box
- [ ] collect the band bbox points into all list, also 
- [ ] show in holoview with the linesegments and its bands


In [ ]:
# test grid class
from grid import CartesianGrid
from sdfs import sdUnitHline, sdStar1, eval_sdf
from functools import partial
xs = np.linspace(-1,1,10)
ys = np.linspace(-1,1,10)[::-1]
zz = eval_sdf(xs, ys, sdStar1)

In [ ]:
%opts Image [padding=0.1, colorbar=False]
zz.shape
hv.Image((xs,ys,zz)) +  hv.Image(zz, bounds=(-1,-1,1,1))

In [ ]:
# create a grid
g = CartesianGrid(xs, ys)
g

In [ ]:
zfunc = partial(eval_sdf, sdFunc=sdStar1)
g.reevaluate(zfunc)
g.hvplot()

In [ ]:
# both way are in sync with Cartesian coordinate system
g2 = CartesianGrid(xs,ys,zz)
g2.hvplot()

Let's check its methods to get/set values at (x,y) cartesian points 

In [ ]:
g.xy2ij(-1,-1)

In [ ]:
c,r = g.xy2cr(0.8,-1)
c,r

In [ ]:
g.get_value(-1,-1)

In [ ]:
g.set_value(-1,-1,0)
g.hvplot()

Great, things look consistent now!

---
Modified: Jul 29, 2019

## LevelSet Evolution


In [ ]:
import calculus as calc
class LSEvolver(CartesianGrid):
    
    def __init__(self, xs, ys, data=None, t=0):
        super().__init__(xs, ys, data)
        self.time = t
        
    def propagate(self, F, dt):
        """
        Equation 4.8 and 4.20
        For stability in computing the spatial gradients, use Eqn. 4.33
        """
        # todo: depending on the order of F we use different spatial difference method
        
        dxb, dxf, dyb, dyf = calc.
        S = np.sign(F)
        
        dx = np.maximum(S*dxb, -S*dxf)
        dy = np.maximum(S*dyb, -S*dyf)
        
        dmag = np.sqrt(dx**2 + dy**2)
        
        # update phi
        self.grid -= dt* dmag * F
        
        # update time
        self.t += dt
        
    def advect(self, V, dt):
        """
        Args:
        - V (ndarray of shape (w,h,2)): containing x and y component of the vector
        field
        - dt (float): time step size
        """
        pass
    
    def reinit(self, method='sweep'):
        """
        Reset current grid (phi function) to satisfy Eikonal equality
        in Eqn. 4.12
        
        - method 
            - 'pde': solve eqn. 4.37 with current grid, until steady state
            - 'fmm': fast marching method
            - 'sweep' (default): paper [88]
            - 'exact': paper [64]
            
            Default is 'sweep'
        """
        pass

    def get_gradient(self, to_switch=True):
        return gradient(self.grid, to_switch)
    
    def get_curvature(self):
        return curvature(self.grid)

                   